In [1]:
import datatable as dt
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt

df = dt.fread('winequality-red.csv').to_pandas()

In [2]:
df.isnull().sum().sum()
# The data set is clean

0

In [3]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
data = df.copy().to_numpy()

In [5]:
inputs = torch.from_numpy(data[:, :-1]).type(torch.float)
targets = torch.from_numpy(data[:, -1]).type(torch.float)
targets = torch.reshape(targets, (-1, 1))

In [6]:
from torch.utils.data import TensorDataset, random_split, DataLoader

dataset = TensorDataset(inputs, targets)
train_ratio = 0.8
train_size = int(len(dataset) * train_ratio)
test_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, test_size])

In [7]:
from torch.utils.data.dataloader import DataLoader

batch_size = 32

# Training sampler and data loader
train_loader = DataLoader(train_ds, batch_size, shuffle=True)

val_loader = DataLoader(val_ds, batch_size)

In [8]:
import torch.nn as nn
import torch.optim as optim

input_size = len(df.columns) - 1
output_size = 1


class My_Model(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, xb):
        return self.linear(xb)

    def training_step(self, batch):
        inputs, targets = batch
        # Generate prediction
        y_pred = self(inputs)
        # Calculate loss
        loss = nn.MSELoss(y_pred, targets)
        return loss

    def validation_step(self, batch):
        inputs, targets = batch
        # Generate prediction
        y_pred = self(inputs)
        # Calculate loss
        loss = nn.MSELoss(y_pred, targets)
        return {'val_loss': loss.detach()}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()  # Combine losses
        return {'val_loss': epoch_loss.item()}

    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 100th epoch
        if (epoch + 1) % (num_epochs // 10) == 0:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch + 1, result['val_loss']))

In [9]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)


def fit(epochs, lr, model, train_loader, val_loader, opt_func=optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

In [10]:
model = My_Model(input_size, output_size)
epochs = 500
lr = 1e-5
history = fit(epochs, lr, model, train_loader, val_loader)

RuntimeError: Boolean value of Tensor with more than one value is ambiguous